In [5]:
%pip install fastapi pydantic toml

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_cn.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


你被一位富商雇佣，调查阿卡姆郊外的一座闹鬼的豪宅。
传说这座豪宅被诅咒，任何进入的人都不会再回来。
你已经得到了豪宅的地图和一些基本装备，以帮助你的调查。
你的目标是找出诅咒背后的真相，并结束它。

你站在闹鬼的豪宅前。
这座豪宅古老而破旧，墙上爬满了爬山虎，窗户被钉上了木板。
前门微微敞开，邀请你进入。

你要做什么？

possible actions:
1. 进入豪宅。
2. 拍摄豪宅外部的照片。
3. 与管家交谈。(可能需要一些说服力，以便让你获取更多信息)


I select option 1. TURN: 1 


你深呼吸一口气，决定进入这座诅咒的豪宅。门吱嘎作响，随着你的推动缓缓开启，仿佛是对你的到来表示欢迎。门后是一个昏暗的大厅，一盏吊灯发出微弱的光芒，四周的空气中弥漫着一种让人不安的气息。

正当你准备进一步探索大厅时，你听到了餐厅方向传来的低沉咆哮声。你意识到，这可能是某种动物，而且听起来并不友好。你小心翼翼地朝声音的方向走去，突然，一只巨大的、满是斑点的狗以惊人的速度向你冲来，它的眼睛里闪烁着恶意的光芒。

你必须迅速做出反应来避开这次攻击。

skill: dodge
difficulty: medium

I am making a skill check using DODGE against a difficulty of MEDIUM.
And I rolled a 36 for a result of SUCCESS. TURN: 2 


你敏捷地一跃而起，巧妙地避开了这只狂暴狗的攻击。它呼啸着飞过你的头顶，撞击在身后的墙上，随后迅速调转方向，准备再次发起攻击。你意识到，这只狗的攻击力非常强大，而且它看起来并不会轻易放弃。

在这紧急的情况下，你需要迅速做出决定。

possible actions:
1. 尝试再次躲避，并寻找逃跑的路线。
2. 用身边的物品（如椅子或桌子）试图阻挡或打晕这只狗。
3. 尝试安抚这只狗，看看它是否会冷静下来。

I select option 3. TURN: 3 


你决定尝试安抚这只看起来狂暴的狗，希望能够找到和它和平相处的方法。你小心翼翼地放低你的声音，尽量让自己看起来不那么威胁，开始轻声地对它说话，希望能够安抚它的情绪。

尽管这是一个冒险的选择，但你意识到，如果成功了，你不仅避免了潜在的伤害，还可能发现这只狗背后隐藏的秘密或者它的存在可能指示着豪宅中的某种线索。

skill: psychology
difficulty: hard

I am making a skill check using PSYCHOLOGY against a difficulty of HARD.
And I rolled a 22 for a result of SUCCESS. TURN: 4 


你的冷静和直觉让你成功地安抚了这只原本狂暴的狗。随着你的声音渐渐安抚它，你注意到它那充满敌意的眼神开始软化，尾巴也缓缓摇动起来。最终，它不再显示出攻击的意图，反而是用鼻子轻轻地触碰你的手，似乎在表示感谢你没有选择用暴力对待它。

这时，你注意到它的颈部挂着一个破旧的项圈，上面有些字迹，但因为磨损已经难以辨认。这只狗似乎对豪宅很熟悉，它开始向一扇半开的门引导你，那扇门通往豪宅的深处。

你跟随这只狗，走进了一条长长的走廊，走廊的尽头有一扇通向地下室的门。门半敞开，里面黑暗而深邃。你听到了来自地下的声音，那是一种低沉而又不寻常的嗡嗡声，似乎隐藏着某种秘密。

正当你准备探索地下室时，你意识到这个回合即将结束。你的心跳加速，知道豪宅中的秘密即将被揭开。

summary: 成功安抚了狂暴的狗之后，你发现这只狗似乎对豪宅中的某些部分非常熟悉，并且引导你到达了通往地下室的门口。嗡嗡声的源头就在地下室内，你正准备揭开豪宅中隐藏的秘密。